In [1]:
%load_ext bigdata
%hive_start
%timeout 300

In [2]:
!hdfs dfs -rm -r -f /tmp/output

Deleted /tmp/output


In [3]:
!hdfs dfs -mkdir /tmp/output

In [4]:
!hdfs dfs -copyFromLocal data.tsv /tmp/output/

In [5]:
%%hive
DROP TABLE IF EXISTS t0;
CREATE TABLE t0 (
    c1 STRING,
    c2 ARRAY<CHAR(1)>, 
    c3 MAP<STRING, INT>
    )
    ROW FORMAT DELIMITED 
        FIELDS TERMINATED BY '\t'
        COLLECTION ITEMS TERMINATED BY ','
        MAP KEYS TERMINATED BY '#'
        LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'data.tsv' INTO TABLE t0;

DROP TABLE IF EXISTS t0;
OK
Time taken: 11.139 seconds
CREATE TABLE t0 (
    c1 STRING,
    c2 ARRAY<CHAR(1)>, 
    c3 MAP<STRING, INT>
    )
    ROW FORMAT DELIMITED 
        FIELDS TERMINATED BY '\t'
        COLLECTION ITEMS TERMINATED BY ','
        MAP KEYS TERMINATED BY '#'
        LINES TERMINATED BY '\n';
OK
Time taken: 0.724 seconds
LOAD DATA LOCAL INPATH 'data.tsv' INTO TABLE t0;
Loading data to table default.t0
OK
Time taken: 1.228 seconds


In [25]:
%%hive

DROP TABLE IF EXISTS resultado;

CREATE TABLE resultado
AS
SELECT  c1,
        SIZE (c2),
        SIZE (c3)
FROM t0;

INSERT OVERWRITE DIRECTORY '/tmp/output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT * FROM resultado;

DROP TABLE IF EXISTS resultado;
OK
Time taken: 0.011 seconds
CREATE TABLE resultado
AS
SELECT  c1,
        SIZE (c2),
        SIZE (c3)
FROM t0;
Query ID = root_20200119013103_dcf3c87d-8a26-4758-b08e-86471f0ef40b
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1579385130637_0024, Tracking URL = http://282e0d2d9a5d:8088/proxy/application_1579385130637_0024/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1579385130637_0024
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 0
2020-01-19 01:31:10,576 Stage-1 map = 0%,  reduce = 0%
2020-01-19 01:31:15,954 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 2.24 sec
MapReduce Total cumulative CPU time: 2 seconds 240 msec
Ended Job = job_1579385130637_0024
Stage-4 is selected by condition resolver.
Stage-3 is filtered out by condition resolver.
Stage-5 is filtered out by condition resolver.
Moving data to directory hdfs://0.0.0.0

In [26]:
%%hive
SELECT * FROM resultado LIMIT 5;

SELECT * FROM resultado LIMIT 5;
OK
E	3	5
A	3	4
B	4	4
A	2	4
C	4	4
Time taken: 0.212 seconds, Fetched: 5 row(s)


In [27]:
!hdfs dfs -copyToLocal /tmp/output output;

In [28]:
%%hive
DROP TABLE resultado;

DROP TABLE resultado;
OK
Time taken: 0.064 seconds


In [29]:
%hive_quit